In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import firls
from firls.tests import simulate
import datetime

C:\Users\jchar\OneDrive\Documents\firls\firls\ccd.py:52: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, A))
  beta = X.T @ y / sum_sq_X.reshape(p, 1)
C:\Users\jchar\OneDrive\Documents\firls\firls\ccd.py:54: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, A))
  XtX = X.T @ X
C:\Users\jchar\OneDrive\Documents\firls\firls\ccd.py:55: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, A))
  Xty = X.T @ y
c:\users\jchar\anaconda3\lib\site-packages\numba\typing\npydecl.py:967: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, A))
  warnings.warn(NumbaPerformanceWarning(msg))
c:\users\jchar\anaconda3\lib\site-packages\numba\typing\npydecl.py:967: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A)

In [6]:

family = "poisson"
sglm = firls.sklearn.GLM(family=family, fit_intercept=False)

for i in [80]:
    y, X, true_beta = simulate.simulate_supervised_poisson(100000, i)
    start = datetime.datetime.now()
    sglm = firls.sklearn.GLM(family=family, fit_intercept=False,solver="ccd")
    sglm.fit(X, y)
    end = datetime.datetime.now()
    total_time  = (end - start).total_seconds()  
    print(total_time)


[[  9.92099965e-02]
 [ -1.03858289e-01]
 [ -2.33066016e-03]
 [  9.73679865e-02]
 [ -1.04897785e-01]
 [  1.02253617e-01]
 [ -9.78047706e-02]
 [  7.80342847e-04]
 [ -1.48138571e-03]
 [  9.90767030e-02]
 [  1.02341591e-01]
 [ -1.53637782e-03]
 [ -1.02701208e-01]
 [ -8.08547011e-05]
 [ -9.88698281e-04]
 [ -7.24403058e-04]
 [ -3.81526559e-03]
 [ -3.59325517e-04]
 [  2.01702322e-03]
 [ -8.57858823e-04]
 [ -1.04567058e-01]
 [  9.86404743e-02]
 [ -1.51005044e-05]
 [  9.78376846e-02]
 [  1.00377727e-01]
 [ -1.24950975e-03]
 [ -1.02536424e-01]
 [ -4.47345886e-03]
 [  1.02414676e-01]
 [ -3.04164347e-01]
 [ -1.04112957e-01]
 [  1.04976903e-01]
 [ -9.83618251e-02]
 [ -3.55108593e-03]
 [ -1.26854872e-03]
 [  8.57720219e-04]
 [  3.96431870e-04]
 [  4.26959161e-03]
 [  1.00132318e-01]
 [ -1.01610301e-01]
 [  1.01600998e-01]
 [  3.22403649e-03]
 [ -1.63182244e-04]
 [ -2.00169987e-01]
 [ -9.99420844e-02]
 [ -1.01870979e-01]
 [ -1.04553611e-01]
 [  1.02158347e-01]
 [  1.01026482e-01]
 [ -4.94873416e-03]


In [42]:
from firls.tests.simulate import (
    simulate_supervised_gaussian,
)
import numpy as np
import pytest

def add_constant(data):
    """add constant to the data.
    """
    n, p = data.shape
    x = np.zeros((n, p + 1))
    x[:, 1:] = data
    x[:, 0] = 1
    return x


def soft_threshold(x, s):
    """Soft thresholding operator.
    """
    return np.sign(x) * np.maximum(np.abs(x) - s, 0)

def ccd_pwls(
    X,
    y,
    W=None,
    fit_intercept=False,
    lambda_l1=0.0,
    lambda_l2=0.0,
    bounds=None,
    max_iters=1000,
    tol=1e-20,
):
    """Coordinate descent algorithm for penalized weighted least squared. Please respect the signature."""
    if fit_intercept:
        X = add_constant(X)
    n, p = X.shape

    if W is None:
        sum_sq_X = np.sum(X ** 2, 0)
    else:
        sum_sq_X = np.sum((X ** 2) * W, 0)
        X = X * W ** 0.5
        y = y * W ** 0.5

    beta = X.T @ y / sum_sq_X.reshape(p, 1)
    beta_old = beta[:]*0
    XtX = X.T @ X
    Xty = X.T @ y
    active_set = list(range(p))
    h = XtX @ beta

    for i in range(max_iters):
        for j in active_set:

            if len(active_set) == 0:
                beta = beta * 0
                beta_old = beta[:]
                break

            beta_j_old = beta[j]
            rho = Xty[j] - (h[j] - beta_j_old * sum_sq_X[j])
            if (fit_intercept) and (j == 0):
                beta_j_new = rho[0] / sum_sq_X[j]
            else:
                beta_j_new = soft_threshold(rho[0], lambda_l1) / (
                    sum_sq_X[j] + lambda_l2
                )
 
 
            h += (XtX[:, j] * (beta_j_new - beta_j_old)).reshape(-1, 1)
            beta[j] = beta_j_new
            
        if np.sum((beta_old - beta) ** 2) ** 0.5 < tol:
            break
        beta_old = np.copy(beta)

    return beta


y, X, true = simulate_supervised_gaussian(1000,40)
w = ccd_pwls(
    X,
    y.reshape((1000,1)),
    W = None,
    fit_intercept=False,
    lambda_l1=0.0,
    lambda_l2=0.0,
    bounds=None,
    max_iters=10000,
    tol=1e-10,
)


w_cf = np.linalg.inv(X.T@X)@X.T@y
np.testing.assert_almost_equal(w.ravel(), w_cf, 4)
print('pass')

pass


In [7]:

family = "poisson"
sglm = firls.sklearn.GLM(family=family, fit_intercept=False)

for i in [80]:
    y, X, true_beta = simulate.simulate_supervised_poisson(100000, i)
    start = datetime.datetime.now()
    sglm = firls.sklearn.GLM(family=family, fit_intercept=False,solver="inv")
    sglm.fit(X, y)
    end = datetime.datetime.now()
    total_time  = (end - start).total_seconds()  
    print(total_time)


[[  9.81277226e-02]
 [ -1.02549136e-01]
 [ -2.21067421e-03]
 [  9.62295322e-02]
 [ -1.04043765e-01]
 [  1.01396325e-01]
 [ -9.70216104e-02]
 [  5.93799411e-04]
 [ -1.26173525e-03]
 [  9.83026639e-02]
 [  1.01527821e-01]
 [ -1.29998503e-03]
 [ -1.01900345e-01]
 [ -2.67343886e-04]
 [ -8.98538719e-04]
 [ -5.17001911e-04]
 [ -3.68242170e-03]
 [ -3.06504133e-04]
 [  1.60108896e-03]
 [ -9.00275418e-04]
 [ -1.03804231e-01]
 [  9.80266116e-02]
 [ -2.20291479e-04]
 [  9.73113722e-02]
 [  9.98125739e-02]
 [ -1.02523911e-03]
 [ -1.02089816e-01]
 [ -4.42881648e-03]
 [  1.01849712e-01]
 [ -3.03570465e-01]
 [ -1.03891373e-01]
 [  1.04882029e-01]
 [ -9.82408182e-02]
 [ -3.54381818e-03]
 [ -1.24363093e-03]
 [  8.53395304e-04]
 [  4.27556981e-04]
 [  4.28144927e-03]
 [  1.00073711e-01]
 [ -1.01589084e-01]
 [  1.01587698e-01]
 [  3.23762655e-03]
 [ -2.70389614e-04]
 [ -2.00107900e-01]
 [ -9.99260585e-02]
 [ -1.01860434e-01]
 [ -1.04577587e-01]
 [  1.02188911e-01]
 [  1.01089362e-01]
 [ -4.91051446e-03]
